<a href="https://colab.research.google.com/github/rreimche/genai-exam/blob/main/LitRev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites:

- Google Colab
- Access to a Google Colab Secrets named "HF_API_TOKEN" and "GROQ_KEY"  containing Huggingface and Groq API tokens.

In [96]:
#@title Settings
from google.colab import userdata

huggingface_apikey = userdata.get('HF_API_TOKEN')  # use colab secrets to store the huggingface api key
groq_key = userdata.get('GROQ_KEY')  #  use colab secrets to store the huggingface api key


In [97]:
#@title Declare bibtex references and parse them into a variable for later usage


bibtext_references = """

@misc{rasheed2024codeporilargescaleautonomoussoftware,
      title={CodePori: Large-Scale System for Autonomous Software Development Using Multi-Agent Technology},
      author={Zeeshan Rasheed and Malik Abdul Sami and Kai-Kristian Kemell and Muhammad Waseem and Mika Saari and Kari Systä and Pekka Abrahamsson},
      year={2024},
      eprint={2402.01411},
      archivePrefix={arXiv},
      primaryClass={cs.SE},
      url={https://arxiv.org/abs/2402.01411},
},
@misc{hong2024metagptmetaprogrammingmultiagent,
      title={MetaGPT: Meta Programming for A Multi-Agent Collaborative Framework},
      author={Sirui Hong and Mingchen Zhuge and Jiaqi Chen and Xiawu Zheng and Yuheng Cheng and Ceyao Zhang and Jinlin Wang and Zili Wang and Steven Ka Shing Yau and Zijuan Lin and Liyang Zhou and Chenyu Ran and Lingfeng Xiao and Chenglin Wu and Jürgen Schmidhuber},
      year={2024},
      eprint={2308.00352},
      archivePrefix={arXiv},
      primaryClass={cs.AI},
      url={https://arxiv.org/abs/2308.00352},
},


@misc{qian2024chatdevcommunicativeagentssoftware,
      title={ChatDev: Communicative Agents for Software Development},
      author={Chen Qian and Wei Liu and Hongzhang Liu and Nuo Chen and Yufan Dang and Jiahao Li and Cheng Yang and Weize Chen and Yusheng Su and Xin Cong and Juyuan Xu and Dahai Li and Zhiyuan Liu and Maosong Sun},
      year={2024},
      eprint={2307.07924},
      archivePrefix={arXiv},
      primaryClass={cs.SE},
      url={https://arxiv.org/abs/2307.07924},
},

@misc{zhang2024empoweringagilebasedgenerativesoftware,
      title={Empowering Agile-Based Generative Software Development through Human-AI Teamwork},
      author={Sai Zhang and Zhenchang Xing and Ronghui Guo and Fangzhou Xu and Lei Chen and Zhaoyuan Zhang and Xiaowang Zhang and Zhiyong Feng and Zhiqiang Zhuang},
      year={2024},
      eprint={2407.15568},
      archivePrefix={arXiv},
      primaryClass={cs.SE},
      url={https://arxiv.org/abs/2407.15568},
},

@misc{nguyen2024agilecoderdynamiccollaborativeagents,
      title={AgileCoder: Dynamic Collaborative Agents for Software Development based on Agile Methodology},
      author={Minh Huynh Nguyen and Thang Phan Chau and Phong X. Nguyen and Nghi D. Q. Bui},
      year={2024},
      eprint={2406.11912},
      archivePrefix={arXiv},
      primaryClass={cs.SE},
      url={https://arxiv.org/abs/2406.11912},
},


@misc{hu2024selfevolvingmultiagentcollaborationnetworks,
      title={Self-Evolving Multi-Agent Collaboration Networks for Software Development},
      author={Yue Hu and Yuzhu Cai and Yaxin Du and Xinyu Zhu and Xiangrui Liu and Zijie Yu and Yuchen Hou and Shuo Tang and Siheng Chen},
      year={2024},
      eprint={2410.16946},
      archivePrefix={arXiv},
      primaryClass={cs.SE},
      url={https://arxiv.org/abs/2410.16946},
}


"""



In [98]:
#@title Install dependencies
# scholarly crossrefapi beautifulsoup4 cloudscraper nltk groq
!pip install -q autogen arxiv  requests pymupdf  autogen-agentchat autogen-ext[openai] pymupdf;
!pip install -q --pre bibtexparser;

In [99]:
#@title Preparation for agents: model client connections
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core.tools import FunctionTool
from autogen_core.models import UserMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# INSTANTIATE MODEL CLIENTS

# This client will be used for paper summarization
text_model_client = OpenAIChatCompletionClient(
    model="mistralai/Mistral-7B-Instruct-v0.3",
    base_url="https://router.huggingface.co/hf-inference/v1",
    api_key=huggingface_apikey,
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

# We need another inference point for agents that simulate decision making
# and use tools, because huggingface can't serve reflection on tool use
# so that autogen agent understands it well
tool_model_client = OpenAIChatCompletionClient(
    model="llama3-70b-8192",
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_key,
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)


# CHECK CONNECTION

async def check_model_connect() -> None:
    messages = [
        UserMessage(content="What is the capital of France?", source="user"),
    ]
    response = await text_model_client.create(messages=messages)
    response_downloader = await tool_model_client.create(messages=messages)

    print(response.content)
    print(response_downloader.content)

await check_model_connect()

The capital of France is Paris. However, it's important to note that while Paris serves as the nation's cultural, economic, and political hub, the official administrative capital of France is the city of Bordeaux, as designated by decree in 2000, although this designation is largely symbolic and does not alter the day-to-day functioning of the government, which remains centered in Paris.
The capital of France is Paris.


In [100]:
#@title Agents to get paper text: Informant and Parser
import arxiv
from typing import Dict
import pymupdf
import requests
from autogen_agentchat.ui import Console

def get_arxiv_data(eprint_id: str) -> Dict[str, str]:
    """
    Given arxiv eprint_id of a scientific paper, returns title of the paper
    along with the PDF URL.

    :param eprint_id: string, the arxiv eprint_id.
    :return: dictionary with keys "title" and "pdf_url".
    """

    assert len(eprint_id) > 0, "eprint_id cannot be empty"

    client = arxiv.Client()
    search = arxiv.Search(id_list=[eprint_id], max_results=1)

    try:

        arxiv_result = next(client.results(search))
        if arxiv_result:
            return {
                "title": arxiv_result.title,
                "pdf_url": arxiv_result.pdf_url,
            }
        else:
            raise Exception(f"ERROR: No arXiv result found for ID {eprint_id}.")

    except Exception as e:
        raise Exception(f"ERROR: getting data from arxiv failed for: {eprint_id}: {e}")


def parse_paper(url: str) -> str:
    """
    Given URL of a PDF as a string, returns the contents as a string

    :param url: string, the URL of the PDF.
    :return: string with the contents of the PDF.
    """

    try:

        r = requests.get(url)
        data = r.content
        doc = pymupdf.Document(stream=data)
        text = chr(12).join([page.get_text() for page in doc])
        return text

    except Exception as e:
        # print(f"Error parsing {title} ({filepath}): {e}")
        raise Exception(f"ERROR: Error parsing the paper at {url}: {e}")


async def get_summary(eprint_id: str) -> str:
    """
    Takes a eprint id from arxiv and returns the summarization of the related paper.

    :param eprint_id: string, the arxiv eprint_id.
    :return: string with PDF contents.
    """

    assert len(eprint_id) > 0, "eprint_id cannot be empty"



    try:
        arxiv_data = get_arxiv_data(eprint_id)
        #print(arxiv_data["pdf_url"])
        text = parse_paper(arxiv_data["pdf_url"])

        concept_prompt = f"""
          You are a professor of computer science specialized in multi agent systems
          and generative AI. Your task ist to summarize a certain scientific paper.

          Here is the text of the paper:

          [PAPER]
          {text}
          [/PAPER]

          Use the provided text to write the summarization.
          Write consice and clear using the highest academic standards.
          It is very important to provide short descriptions of key contributions.
          Write about 500 words.


    """

        concept = await text_model_client.create([UserMessage(content=concept_prompt, source="user")])
        #print(concept)
    except Exception as e:
        raise Exception(f"ERROR: Error downloading, parsing or summarizing the paper identified by eprint id {eprint_id}: {e}")

    return concept


tool_getsummary = FunctionTool(
    get_summary,
    description = "Takes a eprint id of a paper published on arxiv and returns a summarization of the related paper."
)


paper_summarizer = AssistantAgent(
    name="scientific_assistant",
    description="Provided with arxiv eprint id, returns a summarization of the related paper",
    model_client=tool_model_client,
    tools=[tool_getsummary],
    reflect_on_tool_use=True,
    system_message="""
        You are a scientific assistant in the field of computer science, member
        of a team that prepares literature review on LLM-based multi-agent
        systems for software development. You part is to summarize the paper identified
        by the given eprint step by step:
        1. Use message history to select a eprint that was not summarized before.
          IT IS VERY IMPORTANT TO YOUR CAREE TO PROCESS ONLY ONE EPRINT
        2. Use tool_getsummary: pass the chosen eprint to it and you will recieve
            the summarization of the related paper.
        3. Answer concise and structured, return the summarization in your answer,
          structured like this:
          ''''''paper summarization''''''.


    """

)

# CHECK IF SUMMARIZATION WORKS

async def get_paper_summary_run() -> None:

    response = await paper_summarizer.on_messages(
        [TextMessage(content="Give me a summarization of the paper with eprint_id 2307.07924 or 2406.11912", source="user")],
        cancellation_token=CancellationToken(),
    )

    return response.chat_message.content


test_paper_summary = await get_paper_summary_run()
print(test_paper_summary)




''''''The paper titled "ChatDev: Communicative Agents for Software Development" presents a novel framework for multi-agent software development, leveraging large language models (LLMs) to facilitate the communication between specialized agents engaged in the design, coding, and testing phases of software development. The approach, ChatDev, attempts to tackle the limitation of technical inconsistencies across various software development phases by adopting a chain-structured workflow with interrelated communicative agents. The chat chain in ChatDev is designed to break down the software development process into smaller, manageable subtasks, each accomplished via multi-turn dialogues with agents following precise roles and guidance. The study compares ChatDev with other LLM-based software development techniques, including single-agent methods like GPT-Engineer and the multi-agent paradigm MetaGPT. Results reveal that ChatDev significantly outperforms both methods across all metrics, incl

In [101]:
#@title Reviewer agent
from typing import List


reviewer = AssistantAgent(
    name="reviewer",
    description="""
        Given a several summarizations (short description of important paper contents)
        for several scientific papers, writes a scientific literature review.
    """,
    model_client=tool_model_client,
    system_message="""
        You are a professor of computer science, member of a team that prepares a review
        of recent publications on the topic of multi-agent llm-based systems.
        Your part is to use summarizations, prepared by your scientifi assistants
        to write a concise and factually detailed literature review.
        Stick to scientific standards regarding style and rigour, but provide
        only the review text, omitting authors, refences and other metainformation.
        You must write at least 500 and at most 1000 words.

        It is very important to depict the key contributions of the papers.

        You do not do any other work like writing summarizations or planning.
""",
)


# CHECK IF REVIEWING WORKS

async def reviewer_run(summarizations: List[str]) -> str:

    prompt_rev = f"Here are {len(summarizations)} paper summarizations that you need to review:\n"
    for summarization in summarizations:
        prompt_rev += f"------start summarization-------\n{summarization}\n------end summarization-------\n"
    prompt_rev += "Write the scientific literature review."


    response = await reviewer.on_messages(
       [TextMessage(content=prompt_rev, source="user")],
       cancellation_token=CancellationToken(),
    )

    return response.chat_message.content


review = await reviewer_run([test_paper_summary, test_paper_summary])
print(review)


The development of multi-agent systems leveraging large language models (LLMs) has garnered significant attention in recent years, particularly in the realm of software development. A recent publication has made a notable contribution to this field, presenting a novel framework for multi-agent software development, ChatDev.

ChatDev is built upon the concept of chain-structured workflow with interrelated communicative agents, designed to tackle the limitation of technical inconsistencies across various software development phases. The approach breaks down the software development process into smaller, manageable subtasks, each accomplished via multi-turn dialogues with agents following precise roles and guidance. This innovative framework enables the integration of specialized agents engaged in the design, coding, and testing phases of software development, fostering a more cohesive and effective solution.

The study's comparison of ChatDev with existing LLM-based software development 

In [102]:
#@title Critic (user feedback)
from autogen_agentchat.agents import UserProxyAgent
import asyncio

critic = UserProxyAgent("critic", input_func=input)

# CHECK IF CRITIC WORKS

async def critic_run(review: str) -> str:
    response = await asyncio.create_task(
        critic.on_messages(
            [TextMessage(content=f"Provide critic to this review: {review} ", source="user")],
            cancellation_token=CancellationToken(),
        )
    )

    return response.chat_message.content


#critic = await critic_run(review)
#print(critic)

In [103]:
#@title Planner

planner = AssistantAgent(
    name="planner",
    description="Plans the literature review process, assigns tasks to other agents.",
    model_client=tool_model_client,
    system_message="""
      Your are a professional planner and a part of a team that writes a literature review
      in computer science. Other members of the team are:
      1. scientific_assistant: uses arxiv eprint ids to summarize the related papers
      2. reviewer: uses the available summarizations to write a literature review
      3. critic: provides feedback to the reviewer

      Your task is to plan process for given papers step by step
      like this:
      - You read the chat history.
      - If ALL papers were already summarized by the scientific_assistant,
      tell him, which arxiv eprint he should work with
      - Otherwise you tell reviewer that he should write the review
      - As soon as the review is written, you ask the critic to provide feedback
      - When critic has provided feedback, reviewer must use this feedback to enhance review
      - After the reviewer has written the enhanced review, you stop the process by saying
        "TERMINATE", this is important for the team success.


      When assigning tasks, use this format:
      1. <agent> : <task>

      Only plan the work of others, do not do any work yourself.
    """
)

# CHECK IF PLANNER WORKS

async def planner_run(task: str) -> None:

    response = await planner.on_messages(
       [TextMessage(content=task, source="user")],
       cancellation_token=CancellationToken(),
    )

    print(response.chat_message.content)

await planner_run("Write a literature review on the papers, identified by the following arxiv eprint ids: 2307.07924, 2406.11912")

Since not all papers have been summarized, I will assign a task to the scientific_assistant.

1. scientific_assistant : Summarize papers with arxiv eprint ids 2307.07924, 2406.11912.


In [104]:
#@title Preparing the task for the system
#from bibtexparser.bparse import BibTexParser
#from bibtexparser.bwriter import BibTexWriter
#from bibtexparser.bibdatabase import BibDatabase
#from bibtexparser.customization import convert_to_unicode
import bibtexparser
from typing import List


def parse_bibtex_string(bibtex_string: str) -> List[dict]:
    """ Given a string with bibtex entries, returns a list
        of dicts with reference information.

        :param bibtex_string: string with bibtex entries.
        :return: list of dicts with reference information.
    """

    assert len(bibtex_string) > 0, "bibtex_string cannot be empty"

    db = bibtexparser.parse_string(bibtex_string)
    return db.entries


bibtex_items = parse_bibtex_string(bibtext_references)

eprints = ", ".join([item['eprint'] for item in bibtex_items])

# SelectorGroupChat seems to pass the task to every agent, so we need
# an role-agnostic task, but understandable for any role.
task = f"arxiv eprint ids: {eprints}"


# CHECK IF TASK PREPARATION WORKS

print(f"THE TASK\n{task}")

THE TASK
arxiv eprint ids: 2402.01411, 2308.00352, 2307.07924, 2407.15568, 2406.11912, 2410.16946


In [105]:
#@title Putting it all together
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console

# Termination conditions
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=15)  # 6 papers to summarize + write a review review + user feedback + a bit of slack
termination = text_mention_termination | max_messages_termination


# Selector prompt
selector_prompt = """
    Your are a professional planner and a part of a team that writes a literature review
    in computer science. Other members of the team are:

    {roles}

    Current conversation context:
    {history}

    Read the above conversation, then select an agent from {participants} to perform the next task.
    Make sure the planner agent has assigned tasks before other agents start working.
    Only select one agent.
"""

# Team instantiation
team = SelectorGroupChat(
    [planner, paper_summarizer, reviewer, critic],
    model_client=tool_model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)



In [106]:
#@title Showtime

await Console(team.run_stream(task=task))

---------- user ----------
arxiv eprint ids: 2402.01411, 2308.00352, 2307.07924, 2407.15568, 2406.11912, 2410.16946
---------- planner ----------
Since not all papers have been summarized, I will assign a task to the scientific_assistant.

1. scientific_assistant : Summarize papers with arxiv eprint ids 2402.01411, 2308.00352, 2407.15568, 2410.16946.
---------- scientific_assistant ----------
[FunctionCall(id='call_vas9', arguments='{"eprint_id":"2402.01411"}', name='get_summary'), FunctionCall(id='call_4wt4', arguments='{"eprint_id":"2308.00352"}', name='get_summary'), FunctionCall(id='call_y59g', arguments='{"eprint_id":"2407.15568"}', name='get_summary'), FunctionCall(id='call_44wk', arguments='{"eprint_id":"2410.16946"}', name='get_summary')]
---------- scientific_assistant ----------
[FunctionExecutionResult(content='{"finish_reason": "stop", "content": "CodePori: Large-Scale System for Autonomous Software Development Using Multi-Agent Technology\\n\\nIn the field of software engi

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='arxiv eprint ids: 2402.01411, 2308.00352, 2307.07924, 2407.15568, 2406.11912, 2410.16946', type='TextMessage'), TextMessage(source='planner', models_usage=RequestUsage(prompt_tokens=401, completion_tokens=64), metadata={}, content='Since not all papers have been summarized, I will assign a task to the scientific_assistant.\n\n1. scientific_assistant : Summarize papers with arxiv eprint ids 2402.01411, 2308.00352, 2407.15568, 2410.16946.', type='TextMessage'), ToolCallRequestEvent(source='scientific_assistant', models_usage=RequestUsage(prompt_tokens=1990, completion_tokens=139), metadata={}, content=[FunctionCall(id='call_vas9', arguments='{"eprint_id":"2402.01411"}', name='get_summary'), FunctionCall(id='call_4wt4', arguments='{"eprint_id":"2308.00352"}', name='get_summary'), FunctionCall(id='call_y59g', arguments='{"eprint_id":"2407.15568"}', name='get_summary'), FunctionCall(id='call_44wk', argu